In [ ]:
# Necessary installations
'''tensorflow-datasets is the stable version that is released on a cadence of every few months and
tfds-nightly` is released every day and has the latest version of the datasets, and upgrading tensorflow
ensure the version is compatible with the latest version of the dataset.  '''
%pip --no-cache-dir install tensorflow-datasets --user
%pip --no-cache-dir install tfds-nightly --user
%pip --no-cache-dir install --upgrade tensorflow --user

After the above installation, please restart the kernel.

In [2]:
# Necessary imports
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

# Ignore some warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# TODO: Make all other necessary imports.
import numpy as np
import matplotlib.pyplot as plt
import logging
import json

In [4]:
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
tfds.disable_progress_bar()

In [5]:
# Loaded the dataset
dataset, dataset_info = tfds.load('oxford_flowers102', as_supervised = True, with_info = True)

# Split the data into training set, validation set and test set.
training_set, validation_set, test_set = dataset['train'], dataset['validation'], dataset['test']


### Dataset Exploration

In [ ]:
# The number of examples in each set from the dataset info.
items = ['train', 'validation', 'test']
for item in items:
    if item == 'train':
        num_training = dataset_info.splits[item].num_examples
        print(num_training)
    elif item == 'validation':
        num_validation = dataset_info.splits[item].num_examples 
        print(num_validation)
    else:
        num_test = dataset_info.splits[item].num_examples
        print(num_test)

# The number of classes in the dataset from the dataset info.
num_classes = dataset_info.features['label'].num_classes
print(num_classes)

In [ ]:
# The shape and corresponding label of 3 images in the training set.
for image, label in training_set.take(3):
    print(image.shape, label.numpy(), sep='......label: ')


In [ ]:
# Plotting 1 image from the training set. 
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()
    plt.imshow(image, cmap= plt.cm.binary)
    plt.colorbar()
    plt.title('The label of this image is: {}'.format(label))
    plt.show()
  

### Pipeline

In [ ]:
image_size = 224
batch_size = 32

def pipeline_net(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (image_size, image_size))
    image /= 255
    return image, label


training_batches = training_set.cache().shuffle(num_training//2).map(pipeline_net).batch(batch_size).prefetch(1)
validation_batches = validation_set.cache().map(pipeline_net).batch(batch_size).prefetch(1)
testing_batches = test_set.cache().map(pipeline_net).batch(batch_size).prefetch(1)

### Using pre-trained model

In [12]:
# Load the pre-trained model.

pre_trained = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",\
                                   input_shape=(image_size, image_size,3))

pre_trained.trainable = False

In [ ]:
# Training
model = tf.keras.Sequential([
        pre_trained,
        tf.keras.layers.Dense(102, activation = 'softmax')
])

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
'''Since this is a pre-trained model I used lesser epochs, because I will still get better result with it'''
EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

trained_model = model.fit(training_batches,
                                    epochs=EPOCHS,
                                    validation_data=validation_batches,
                                    callbacks=[early_stopping])
    

In [ ]:
# Plot showing the loss and accuracy values achieved during training for the training and validation set.

training_accuracy = trained_model.history['accuracy']
validation_accuracy = trained_model.history['val_accuracy']

training_loss = trained_model.history['loss']
validation_loss = trained_model.history['val_loss']

epochs_range=range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, training_loss, label='Training Loss')
plt.plot(epochs_range, validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Testing the Network


In [ ]:
# TODO: Print the loss and accuracy values achieved on the entire test set.
test_result = model.evaluate(testing_batches)
print(f'test_loss: {test_result[0]}, and test_accuracy : {test_result[1]}')


In [34]:
# Saved the trained model as a Keras model.

model.save('./my_class_model.h5')